Copyright (C) Microsoft Corporation.

# Author a real time web service

**Important**: Make sure the kernel is set to "Your project name myvm" which can be done from the *Kernel* menu under *Change kernel*.

In this notebook, you will be using the LightGBM classifer you trained in the second notebook to prepare the necessary artifacts to opertionalize your model.

In [1]:
import os
from os import path
from glob import iglob
from PIL import Image, ImageOps
import numpy as np
import base64
import json
from io import BytesIO
import sys
import lightgbm as lgb
from azure.storage.blob import BlockBlobService
import shutil

In [2]:
os.environ["KERAS_BACKEND"] = "cntk"
import keras
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input

Using CNTK backend


You will use the AZUREML_NATIVE_SHARE_DIRECTORY to save your operationalization artifacts.

In [3]:
save_path = os.environ['AZUREML_NATIVE_SHARE_DIRECTORY']

You will use the first fail image as a test example to prepare your service.

In [4]:
files_path = path.join(save_path, 'train')
fail_files = sorted(iglob(path.join(files_path, '*fail*.jpg')))

Pick first fail image.

In [5]:
fail_img_path = fail_files[0]
fail_pil_image = Image.open(fail_img_path)
print(fail_pil_image.size)
#fail_pil_image

(500, 374)


In order to transfer the image to call the web service, you will encode the image into string representation and then serialize it into json format.

In [6]:
with open(fail_img_path, 'rb') as file:
  encoded = base64.b64encode(file.read())
body = json.dumps("{}".format(encoded))
body

'"b\'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAoHBwgHBgoICAgLCgoLDhgQDg0NDh0VFhEYIx8lJCIfIiEmKzcvJik0KSEiMEExNDk7Pj4+JS5ESUM8SDc9Pjv/2wBDAQoLCw4NDhwQEBw7KCIoOzs7Ozs7Ozs7Ozs7Ozs7Ozs7Ozs7Ozs7Ozs7Ozs7Ozs7Ozs7Ozs7Ozs7Ozs7Ozs7Ozv/wAARCAF2AfQDASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwDokqVKiSpVr5s9UlWpFqNalWkMeop4pFIqQY9KAAU4UDbS/LSAUU4GmilFADxTxUamng0CY8Y6Gq1zb5BOKsCn4BGDQJOzOZvbIPkFQaz49PW3bkV1U9sM5ArOuYO9aRfc6ITM7YOgFRSRjFTscZHeo296ckdUZFOSMMC

The above string representation will be sent to the web service.It will then be loaded, decoded and converted back into an image by the web service before featurization and prediction steps. Below, you will load this string and perform necessary steps to convert it back to its image representation.

In [7]:
base64ImgString = json.loads(body)

if base64ImgString.startswith('b\''):
    base64ImgString = base64ImgString[2:-1]
base64Img = base64ImgString.encode('utf-8')

decoded_img = base64.b64decode(base64Img)
img_buffer = BytesIO(decoded_img)
imageData = Image.open(img_buffer).convert("RGB")

Now, you will load ResNet50 model and LightGBM classifer to make a prediction on the first fail image.

In [8]:
clf = lgb.Booster(model_file=path.join(save_path,'lightgbm_classifier.model'))
model = ResNet50(include_top=False, input_shape=(224,224,3))
img = ImageOps.fit(imageData, (224, 224), Image.ANTIALIAS)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
pred = model.predict(x).squeeze()  
feat = pred.reshape(1, pred.shape[0])
resp = clf.predict(feat)
resp

92790784/94653016 [============================>.] - ETA: 0s

array([ 0.00011647])

## Create a scoring script

In order to create a web service, you will create a scoring script that will load the models, perform the prediction, and return the result. Azure ML Workbench uses init() and run() functions inside this scoring script for that purpose. The init() function initializes the web service and loads the saved model. The run() function uses the model and the input data to return a prediction which is executed on a scoring call.

You will first define init() and run() functions and test them. Notice that the init() function loads the LightGBM model and the run() function just wraps the above steps to convert the image string received into image format, load ResNet50 model to create features and use the lightGBM model to make a prediction.

In [9]:
def init():
    global clf
    try:
        print("Executing init() method...")
        print("Python version: " + str(sys.version) + ", keras version: " + keras.__version__)
        # Load the model 
        clf = lgb.Booster(model_file='/azureml-share/lightgbm_classifier.model')
    except Exception as e:
        print("Exception in init:")
        print(str(e))
   
def run(inputString):
    try:
        responses = []
        base64ImgString = json.loads(inputString)
        
        if base64ImgString.startswith('b\''):
            base64ImgString = base64ImgString[2:-1]
        base64Img = base64ImgString.encode('utf-8')
            
        # Preprocess the input data
        decoded_img = base64.b64decode(base64Img)
        img_buffer = BytesIO(decoded_img)
        imageData = Image.open(img_buffer).convert("RGB")
           
        # Evaluate the model using the input data
        model = ResNet50(include_top=False, input_shape=(224,224,3))
        img = ImageOps.fit(imageData, (224, 224), Image.ANTIALIAS)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        pred = model.predict(x).squeeze()  
        feat = pred.reshape(1, pred.shape[0])
        resp = clf.predict(feat)
        responses.append(list(resp))
    except Exception as e:
        print("Exception in run:")
        print(str(e))                
    return responses

You will now test your functions. You will see that the same prediction value is produced by the run() function when the same image string is passed.

In [10]:
init()
run(body)

Executing init() method...
Python version: 3.5.2 | packaged by conda-forge | (default, Jan 19 2017, 15:28:33) 
[GCC 4.8.2 20140120 (Red Hat 4.8.2-15)], keras version: 2.0.8


[[0.00011646905475415675]]

Write the scoring script to the folder you created in the 2nd notebook to be used by operationalization later. Observe that the scoring script includes the init() and run() functions and also imports the necessary packages to run those. Also, notice that the path to the model file doesn't have the folder name in the scoring script as the web service will place all the operationalization files in one location.

In [32]:
o16n_path = path.join(save_path,'o16n')
write_path = path.join(o16n_path, 'imgscore.py')

In [33]:
%%writefile $write_path

import os
os.environ["KERAS_BACKEND"] = "cntk"
import keras
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
from keras.applications.resnet50 import ResNet50

import sys, base64, json
from PIL import Image, ImageOps
import lightgbm as lgb
from io import BytesIO
import numpy as np


try:
    import lightgbm as lgb
except OSError as e:
    print(str(e))


def init():
    global clf
    try:
        print("Executing init() method...")
        print("Python version: " + str(sys.version) + ", keras version: " + keras.__version__)
        # Load the model 
        clf = lgb.Booster(model_file='lightgbm_classifier.model')
    except Exception as e:
        print("Exception in init:")
        print(str(e))
   

def run(inputString):
    try:
        responses = []
        base64ImgString = json.loads(inputString)
        
        if base64ImgString.startswith('b\''):
            base64ImgString = base64ImgString[2:-1]
        base64Img = base64ImgString.encode('utf-8')
            
        # Preprocess the input data
        decoded_img = base64.b64decode(base64Img)
        img_buffer = BytesIO(decoded_img)
        imageData = Image.open(img_buffer).convert("RGB")
           
        # Evaluate the model using the input data
        model = ResNet50(include_top=False, input_shape=(224,224,3))
        img = ImageOps.fit(imageData, (224, 224), Image.ANTIALIAS)
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        pred = model.predict(x).squeeze()  
        feat = pred.reshape(1, pred.shape[0])
        resp = clf.predict(feat)
        responses.append(list(resp))
    except Exception as e:
        print("Exception in run:")
        print(str(e))                
    return responses

if __name__ == "__main__":
    init()
    # input data
    img_path = 'fail.0.jpg'
    encoded = None
    with open(img_path, 'rb') as file:
      encoded = base64.b64encode(file.read())
    body = json.dumps("{}".format(encoded))
    resp = run(body)
    print(resp)

Overwriting /azureml-share/o16n/imgscore.py


You will also use the first failure image as sample data to test your web services. Next, you will compy the image into the operationalization folder.

In [34]:
shutil.copyfile(fail_img_path, path.join(o16n_path,path.split(fail_img_path)[1]))

'/azureml-share/o16n/fail.0.jpg'

Check if the image, scoring script and model are in the operationalizaton folder.

In [35]:
!ls $o16n_path

fail.0.jpg  imgscore.py  lightgbm_classifier.model


## Upload operationalization files to blob storage

Now, you will compress the operationalization folder and upload to blob storage. Locate your storage account name and key as described in the Getting Started document of this tutorial and provide your values in the following cell where indicated.

In [36]:
ACCOUNT_NAME = 'Your Storage Account Name Here!!'
ACCOUNT_KEY = 'Your Storage Account Key Here!!'
CONTAINER_NAME = "deploy"
ZIP_FILE = 'o16n.zip'

First, create a container. If the container already exists, the code will return "False".

In [37]:
block_blob_service = BlockBlobService(account_name=ACCOUNT_NAME, account_key=ACCOUNT_KEY)
block_blob_service.create_container(container_name=CONTAINER_NAME, fail_on_exist=False)

False

Check if the container is created.

In [38]:
for container in block_blob_service.list_containers():
    print(container.name)

deploy
images


Compress the operationalization folder and upload to blob.

In [39]:
shutil.make_archive('o16n', 'zip', o16n_path)
block_blob_service.create_blob_from_path(container_name=CONTAINER_NAME,blob_name=ZIP_FILE, file_path=ZIP_FILE) 

Check if the file is uploaded to blob.

In [40]:
for blob in block_blob_service.list_blobs(container_name=CONTAINER_NAME):
    print(blob.name)

o16n.zip


Next, go to the 4th notebook to deploy your model.